In [1]:
import random
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pandas as pd
import sentence_transformers
from sentence_transformers import SentenceTransformer

c:\Users\nabaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
xlsx_file_path = "D:/Tasks & Internships material/RadicalX/Final.xlsx"
df_xlsx = pd.read_excel(xlsx_file_path)

In [10]:
categories = df_xlsx['Category'].unique()

# Let the user choose a category
print("Choose a category:")
for i, category in enumerate(categories):
    print(f"{i + 1}. {category}")

Choose a category:
1. Soft Skills Questions
2. Technical Questions


In [11]:
selected_category_index = int(input("Enter the number of your chosen category: ")) - 1
selected_category = categories[selected_category_index]

# Filter the DataFrame based on the selected category
category_df = df_xlsx[df_xlsx['Category'] == selected_category]

In [12]:
# Ask the user about the difficulty level
difficulty_levels = ["Beginner", "Intermediate", "Expert"]
print("Choose a difficulty level:")
for i, difficulty in enumerate(difficulty_levels):
    print(f"{i + 1}. {difficulty}")

Choose a difficulty level:
1. Beginner
2. Intermediate
3. Expert


In [13]:
selected_difficulty_index = int(input("Enter the number of your chosen difficulty level: ")) - 1
selected_difficulty = difficulty_levels[selected_difficulty_index]

In [14]:
# Filter the DataFrame based on the selected difficulty level
difficulty_df = category_df[category_df['Difficulty'] == selected_difficulty]

# Ask the user about the number of questions
num_questions = int(input("Enter the number of questions you want to be asked: "))

terms_dict = pd.Series(difficulty_df.Answer.values, index=difficulty_df.Question).to_dict()

In [15]:
model_name = "bert-base-nli-mean-tokens"
model = SentenceTransformer(model_name)

def encode(text):
    return model.encode(text, convert_to_tensor=True)

In [16]:
asked_questions = set()

for _ in range(num_questions):
    # Filter out questions that have already been asked in the current session
    remaining_questions = set(terms_dict.keys()) - asked_questions

    if not remaining_questions:
        print("You've already answered all available questions. Exiting.")
        break

    random_term = random.choice(list(remaining_questions))
    answer = terms_dict[random_term]
    asked_questions.add(random_term)

    print(f"\nPlease Answer This Question: {random_term}")
    user_input = input("Your Answer: ")

    answer_vector = encode(answer).unsqueeze(0)
    user_input_vector = encode(user_input).unsqueeze(0)

    similarity = cosine_similarity(answer_vector, user_input_vector)
    print(f"Your Accuracy is: {similarity[0][0]}")
    print(f"Your Answer: {user_input}")
    print(f"Correct Answer: {answer}")



Please Answer This Question: As a user, what strategies do you employ to effectively prioritize tasks in a fast-paced work environment? Can you share a specific example?
Your Accuracy is: 0.5860296487808228
Your Answer: I prioritize acording to the team's dicisons and their opinions
Correct Answer: In a fast-paced work environment, I employ a combination of time management techniques and priority assessment. For instance, I use the Eisenhower Matrix to categorize tasks into urgent and important, helping me focus on what needs immediate attention. An example would be when facing tight deadlines, I prioritize tasks based on their impact on project timelines, ensuring critical milestones are met first.

Please Answer This Question: As a job candidate, how do you actively listen in a professional setting? Provide an example where your active listening skills positively influenced the outcome of a discussion or project.
Your Accuracy is: -0.008456021547317505
Your Answer: i don't give a fu

In [ ]:
# 50-70 > 75%
# 70 - 100 > 100%
# 30 - 50 > 50%